# Deep learning

In this file, we try to use the ANN, combined with other approaches like (de)centralized and seasonality aggregation, for predicting weekly sales. We assume that the data is available for only a single year.

In [1]:
import pandas as pd
sales=pd.read_csv("E:/GitHubActivities/rep1-Analysis on retail demand prediction/data/data_processed.csv")
NW = 52 #number of weeks to be considered in the dataset

# Centralized and decentralized approaches

There are two general approaches to predict the demand of products. The first is to consider a single predictor for all product items, called centralized method, and the other approach is to consider products separately, called decentralized method.


In [2]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import r2_score, mean_squared_error

ANN = keras.models.Sequential([
    keras.layers.Dense(5, activation='relu'),
    keras.layers.Dense(5, activation='relu'),
    keras.layers.Dense(1, activation='LeakyReLU')
])

ANN.compile(optimizer=tf.compat.v1.train.AdamOptimizer(), 
              loss='mean_squared_error',
              metrics=['mean_squared_error'])


res=pd.DataFrame(index=['R2'])

## Structuring the dataset

Before proceeding to the approaches mentioned above, we need to split our data into train and test datasets for each specific product.

In [3]:
skuSet = list(sales.sku.unique())
skuData = {}
colnames = [i for i in sales.columns if i not in ["week","weekly_sales","sku"]]
for i in skuSet:
  df_i = sales[sales.sku == i]
  skuData[i] = {'X': df_i[:NW][colnames].values,
                'y': df_i[:NW]['weekly_sales'].values}
    
X_dict = {}
y_dict = {}

y_test = []
y_train = []

for i in skuSet:
  
  X_train_i,X_test_i = train_test_split(skuData[i]["X"], shuffle=False, train_size=0.7) #split for X
  y_train_i,y_test_i = train_test_split(skuData[i]["y"], shuffle=False, train_size=0.7) #split for y 

  X_dict[i] = {'train': X_train_i, 'test': X_test_i} #filling dictionary
  y_dict[i] = {'train': y_train_i, 'test': y_test_i}

  y_test += list(y_test_i) 
  y_train += list(y_train_i) 

## Centralized method

 Once the train and test datasets are created for each product, we combine them to deploy our centralized solution method and evaluate it.

In [4]:
X_cen_train = X_dict[skuSet[0]]['train'] #initialization with item 0
X_cen_test = X_dict[skuSet[0]]['test']

for i in skuSet[1:]: #Iteration over items
    X_cen_train = np.concatenate((X_cen_train, X_dict[i]['train']), axis = 0) #Bringing together the training set
    X_cen_test = np.concatenate((X_cen_test, X_dict[i]['test']), axis = 0)

X_cen_train = np.asarray(X_cen_train)
X_cen_test = np.asarray(X_cen_test)

y_test = np.asarray(y_test)
y_train = np.asarray(y_train)

model_cen = ANN
model_cen.fit(X_cen_train,y_train,batch_size=20,epochs=500,validation_split=0)

y_pred = model_cen.predict(X_cen_test)

print('Centralized method with linear regression R2:',
      round(r2_score(y_test, np.array(y_pred)),3))  
print('Centralized method with linear regression MSE:',
      round(mean_squared_error(y_test, np.array(y_pred)),3))

res['Centralized(DL)']=[r2_score(y_test, model_cen.predict(X_cen_test))]

Epoch 1/500
80/80 [==============================] - 1s 2ms/step - loss: 82728.7812 - mean_squared_error: 82728.7812
Epoch 2/500
80/80 [==============================] - 0s 2ms/step - loss: 81815.7266 - mean_squared_error: 81815.7266
Epoch 3/500
80/80 [==============================] - 0s 1ms/step - loss: 81641.3047 - mean_squared_error: 81641.3047
Epoch 4/500
80/80 [==============================] - 0s 2ms/step - loss: 81494.3828 - mean_squared_error: 81494.3828
Epoch 5/500
80/80 [==============================] - 0s 2ms/step - loss: 81329.7031 - mean_squared_error: 81329.7031
Epoch 6/500
80/80 [==============================] - 0s 2ms/step - loss: 81126.9609 - mean_squared_error: 81126.9609
Epoch 7/500
80/80 [==============================] - 0s 2ms/step - loss: 80878.8047 - mean_squared_error: 80878.8125
Epoch 8/500
80/80 [==============================] - 0s 2ms/step - loss: 80573.6953 - mean_squared_error: 80573.6953
Epoch 9/500
80/80 [==============================] - 0s 2ms/step

80/80 [==============================] - 0s 1ms/step - loss: 44288.9492 - mean_squared_error: 44288.9492
Epoch 140/500
80/80 [==============================] - 0s 1ms/step - loss: 44145.1211 - mean_squared_error: 44145.1211
Epoch 141/500
80/80 [==============================] - 0s 1ms/step - loss: 44076.8633 - mean_squared_error: 44076.8633
Epoch 142/500
80/80 [==============================] - 0s 1ms/step - loss: 43910.5156 - mean_squared_error: 43910.5156
Epoch 143/500
80/80 [==============================] - 0s 1ms/step - loss: 43828.9531 - mean_squared_error: 43828.9531
Epoch 144/500
80/80 [==============================] - 0s 1ms/step - loss: 43760.4688 - mean_squared_error: 43760.4688
Epoch 145/500
80/80 [==============================] - 0s 1ms/step - loss: 43633.9141 - mean_squared_error: 43633.9141
Epoch 146/500
80/80 [==============================] - 0s 1ms/step - loss: 43491.5156 - mean_squared_error: 43491.5156
Epoch 147/500
80/80 [==============================] - 0s 1ms/

Epoch 208/500
80/80 [==============================] - 0s 1ms/step - loss: 37256.3242 - mean_squared_error: 37256.3242
Epoch 209/500
80/80 [==============================] - 0s 1ms/step - loss: 37217.7539 - mean_squared_error: 37217.7500
Epoch 210/500
80/80 [==============================] - 0s 1ms/step - loss: 37092.5898 - mean_squared_error: 37092.5898
Epoch 211/500
80/80 [==============================] - 0s 1ms/step - loss: 37028.7617 - mean_squared_error: 37028.7617
Epoch 212/500
80/80 [==============================] - 0s 1ms/step - loss: 36931.9141 - mean_squared_error: 36931.9141
Epoch 213/500
80/80 [==============================] - 0s 1ms/step - loss: 36873.2031 - mean_squared_error: 36873.2031
Epoch 214/500
80/80 [==============================] - 0s 2ms/step - loss: 36751.4102 - mean_squared_error: 36751.4102
Epoch 215/500
80/80 [==============================] - 0s 1ms/step - loss: 36644.9414 - mean_squared_error: 36644.9375
Epoch 216/500
80/80 [===========================

Epoch 345/500
80/80 [==============================] - 0s 1ms/step - loss: 29313.4727 - mean_squared_error: 29313.4727
Epoch 346/500
80/80 [==============================] - 0s 1ms/step - loss: 29259.4160 - mean_squared_error: 29259.4160
Epoch 347/500
80/80 [==============================] - 0s 1ms/step - loss: 29134.6035 - mean_squared_error: 29134.6035
Epoch 348/500
80/80 [==============================] - 0s 1ms/step - loss: 29183.7520 - mean_squared_error: 29183.7520
Epoch 349/500
80/80 [==============================] - 0s 1ms/step - loss: 29087.3691 - mean_squared_error: 29087.3691
Epoch 350/500
80/80 [==============================] - 0s 1ms/step - loss: 29106.7344 - mean_squared_error: 29106.7344
Epoch 351/500
80/80 [==============================] - 0s 1ms/step - loss: 29101.0117 - mean_squared_error: 29101.0117
Epoch 352/500
80/80 [==============================] - 0s 1ms/step - loss: 28961.9395 - mean_squared_error: 28961.9395
Epoch 353/500
80/80 [===========================

Epoch 482/500
80/80 [==============================] - 0s 1ms/step - loss: 26590.3809 - mean_squared_error: 26590.3809
Epoch 483/500
80/80 [==============================] - 0s 1ms/step - loss: 26576.4570 - mean_squared_error: 26576.4570
Epoch 484/500
80/80 [==============================] - 0s 1ms/step - loss: 26486.9551 - mean_squared_error: 26486.9551
Epoch 485/500
80/80 [==============================] - 0s 1ms/step - loss: 26497.9492 - mean_squared_error: 26497.9492
Epoch 486/500
80/80 [==============================] - 0s 1ms/step - loss: 26480.9316 - mean_squared_error: 26480.9316
Epoch 487/500
80/80 [==============================] - 0s 1ms/step - loss: 26513.2676 - mean_squared_error: 26513.2676
Epoch 488/500
80/80 [==============================] - 0s 1ms/step - loss: 26535.4922 - mean_squared_error: 26535.4922
Epoch 489/500
80/80 [==============================] - 0s 1ms/step - loss: 26407.8867 - mean_squared_error: 26407.8867
Epoch 490/500
80/80 [===========================

## Decentralized mehod

In this subsection, a dictionary of prediction mehods is created for each productn and the total accuracy of it is caculated then.


In [5]:
y_pred = []
skuModels = {}

for i in skuSet:
 #one model for each item, fitted on training set
 model_i = ANN
 skuModels[i] = model_i.fit(X_dict[i]['train'],y_dict[i]['train'],batch_size=20,epochs=50,validation_split=0)

 #compute and concatenate prediction of the model i on item i
 y_pred += list(model_i.predict(X_dict[i]['test']))


#computing overall performance metrics on y_pred and y_test:
print('Decentralized method with linear regression R2:',round(r2_score(y_test, np.array(y_pred)),3))
print('Decentralized method with linear regression MSE:', round(mean_squared_error(y_test, np.array(y_pred)),3))

res['Decentralized(DL)']=[r2_score(y_test, np.array(y_pred))]

Epoch 1/50
2/2 [==============================] - 0s 5ms/step - loss: 30263.2852 - mean_squared_error: 30263.2852
Epoch 2/50
2/2 [==============================] - 0s 4ms/step - loss: 29562.4922 - mean_squared_error: 29562.4922
Epoch 3/50
2/2 [==============================] - 0s 5ms/step - loss: 28488.5527 - mean_squared_error: 28488.5527
Epoch 4/50
2/2 [==============================] - 0s 2ms/step - loss: 26867.7051 - mean_squared_error: 26867.7051
Epoch 5/50
2/2 [==============================] - 0s 8ms/step - loss: 25218.5547 - mean_squared_error: 25218.5547
Epoch 6/50
2/2 [==============================] - 0s 6ms/step - loss: 23526.8516 - mean_squared_error: 23526.8516
Epoch 7/50
2/2 [==============================] - 0s 3ms/step - loss: 21817.2598 - mean_squared_error: 21817.2598
Epoch 8/50
2/2 [==============================] - 0s 3ms/step - loss: 20011.0215 - mean_squared_error: 20011.0215
Epoch 9/50
2/2 [==============================] - 0s 6ms/step - loss: 18640.6641 - mean_

Epoch 47/50
2/2 [==============================] - 0s 4ms/step - loss: 304.8926 - mean_squared_error: 304.8926
Epoch 48/50
2/2 [==============================] - 0s 3ms/step - loss: 304.1357 - mean_squared_error: 304.1357
Epoch 49/50
2/2 [==============================] - 0s 2ms/step - loss: 304.0466 - mean_squared_error: 304.0466
Epoch 50/50
2/2 [==============================] - 0s 4ms/step - loss: 305.4896 - mean_squared_error: 305.4896
Epoch 1/50
2/2 [==============================] - 0s 4ms/step - loss: 2810.7900 - mean_squared_error: 2810.7900
Epoch 2/50
2/2 [==============================] - 0s 3ms/step - loss: 2735.5557 - mean_squared_error: 2735.5557
Epoch 3/50
2/2 [==============================] - 0s 4ms/step - loss: 2607.0466 - mean_squared_error: 2607.0466
Epoch 4/50
2/2 [==============================] - 0s 3ms/step - loss: 2442.1033 - mean_squared_error: 2442.1033
Epoch 5/50
2/2 [==============================] - 0s 3ms/step - loss: 2289.6858 - mean_squared_error: 2289.6

Epoch 44/50
2/2 [==============================] - 0s 3ms/step - loss: 527.5621 - mean_squared_error: 527.5621
Epoch 45/50
2/2 [==============================] - 0s 3ms/step - loss: 526.1721 - mean_squared_error: 526.1720
Epoch 46/50
2/2 [==============================] - 0s 3ms/step - loss: 525.6332 - mean_squared_error: 525.6332
Epoch 47/50
2/2 [==============================] - 0s 3ms/step - loss: 525.0788 - mean_squared_error: 525.0788
Epoch 48/50
2/2 [==============================] - 0s 3ms/step - loss: 524.5138 - mean_squared_error: 524.5138
Epoch 49/50
2/2 [==============================] - 0s 4ms/step - loss: 523.8647 - mean_squared_error: 523.8647
Epoch 50/50
2/2 [==============================] - 0s 2ms/step - loss: 523.2258 - mean_squared_error: 523.2258
Epoch 1/50
2/2 [==============================] - 0s 4ms/step - loss: 1293.3209 - mean_squared_error: 1293.3209
Epoch 2/50
2/2 [==============================] - 0s 3ms/step - loss: 1257.1155 - mean_squared_error: 1257.1155

2/2 [==============================] - 0s 4ms/step - loss: 1511.1305 - mean_squared_error: 1511.1305
Epoch 41/50
2/2 [==============================] - 0s 3ms/step - loss: 1431.4386 - mean_squared_error: 1431.4386
Epoch 42/50
2/2 [==============================] - 0s 2ms/step - loss: 1457.8418 - mean_squared_error: 1457.8418
Epoch 43/50
2/2 [==============================] - 0s 3ms/step - loss: 1388.4585 - mean_squared_error: 1388.4585
Epoch 44/50
2/2 [==============================] - 0s 3ms/step - loss: 1459.2880 - mean_squared_error: 1459.2880
Epoch 45/50
2/2 [==============================] - 0s 3ms/step - loss: 1384.4060 - mean_squared_error: 1384.4060
Epoch 46/50
2/2 [==============================] - 0s 3ms/step - loss: 1436.7031 - mean_squared_error: 1436.7031
Epoch 47/50
2/2 [==============================] - 0s 3ms/step - loss: 1348.2997 - mean_squared_error: 1348.2997
Epoch 48/50
2/2 [==============================] - 0s 3ms/step - loss: 1456.2305 - mean_squared_error: 1456.

2/2 [==============================] - 0s 3ms/step - loss: 11162.0811 - mean_squared_error: 11162.0811
Epoch 36/50
2/2 [==============================] - 0s 3ms/step - loss: 11181.0498 - mean_squared_error: 11181.0498
Epoch 37/50
2/2 [==============================] - 0s 3ms/step - loss: 11126.2109 - mean_squared_error: 11126.2119
Epoch 38/50
2/2 [==============================] - 0s 4ms/step - loss: 11158.2080 - mean_squared_error: 11158.2070
Epoch 39/50
2/2 [==============================] - 0s 3ms/step - loss: 11091.0996 - mean_squared_error: 11091.0996
Epoch 40/50
2/2 [==============================] - 0s 3ms/step - loss: 11054.1836 - mean_squared_error: 11054.1836
Epoch 41/50
2/2 [==============================] - 0s 3ms/step - loss: 11125.3916 - mean_squared_error: 11125.3916
Epoch 42/50
2/2 [==============================] - 0s 3ms/step - loss: 11024.3496 - mean_squared_error: 11024.3506
Epoch 43/50
2/2 [==============================] - 0s 3ms/step - loss: 11053.7246 - mean_squ

...

2/2 [==============================] - 0s 3ms/step - loss: 516.7801 - mean_squared_error: 516.7801
Epoch 43/50
2/2 [==============================] - 0s 3ms/step - loss: 517.5618 - mean_squared_error: 517.5618
Epoch 44/50
2/2 [==============================] - 0s 4ms/step - loss: 517.6443 - mean_squared_error: 517.6443
Epoch 45/50
2/2 [==============================] - 0s 3ms/step - loss: 514.8027 - mean_squared_error: 514.8027
Epoch 46/50
2/2 [==============================] - 0s 3ms/step - loss: 513.4464 - mean_squared_error: 513.4464
Epoch 47/50
2/2 [==============================] - 0s 4ms/step - loss: 512.3069 - mean_squared_error: 512.3069
Epoch 48/50
2/2 [==============================] - 0s 4ms/step - loss: 511.5635 - mean_squared_error: 511.5635
Epoch 49/50
2/2 [==============================] - 0s 3ms/step - loss: 511.1505 - mean_squared_error: 511.1505
Epoch 50/50
2/2 [==============================] - 0s 3ms/step - loss: 510.0754 - mean_squared_error: 510.0754
Epoch 1/50
2/

2/2 [==============================] - 0s 4ms/step - loss: 3605.6172 - mean_squared_error: 3605.6172
Epoch 35/50
2/2 [==============================] - 0s 5ms/step - loss: 3523.6467 - mean_squared_error: 3523.6467
Epoch 36/50
2/2 [==============================] - 0s 3ms/step - loss: 3441.1396 - mean_squared_error: 3441.1396
Epoch 37/50
2/2 [==============================] - 0s 3ms/step - loss: 3364.2271 - mean_squared_error: 3364.2271
Epoch 38/50
2/2 [==============================] - 0s 3ms/step - loss: 3285.0417 - mean_squared_error: 3285.0417
Epoch 39/50
2/2 [==============================] - 0s 4ms/step - loss: 3214.1953 - mean_squared_error: 3214.1953
Epoch 40/50
2/2 [==============================] - 0s 3ms/step - loss: 3139.1499 - mean_squared_error: 3139.1499
Epoch 41/50
2/2 [==============================] - 0s 3ms/step - loss: 3068.6719 - mean_squared_error: 3068.6719
Epoch 42/50
2/2 [==============================] - 0s 4ms/step - loss: 2997.7119 - mean_squared_error: 2997.

2/2 [==============================] - 0s 3ms/step - loss: 258.2462 - mean_squared_error: 258.2462
Epoch 32/50
2/2 [==============================] - 0s 3ms/step - loss: 256.4173 - mean_squared_error: 256.4173
Epoch 33/50
2/2 [==============================] - 0s 4ms/step - loss: 254.6974 - mean_squared_error: 254.6974
Epoch 34/50
2/2 [==============================] - 0s 3ms/step - loss: 253.3004 - mean_squared_error: 253.3004
Epoch 35/50
2/2 [==============================] - 0s 3ms/step - loss: 251.6682 - mean_squared_error: 251.6682
Epoch 36/50
2/2 [==============================] - 0s 2ms/step - loss: 250.1825 - mean_squared_error: 250.1825
Epoch 37/50
2/2 [==============================] - 0s 2ms/step - loss: 248.8044 - mean_squared_error: 248.8044
Epoch 38/50
2/2 [==============================] - 0s 3ms/step - loss: 247.6982 - mean_squared_error: 247.6982
Epoch 39/50
2/2 [==============================] - 0s 3ms/step - loss: 246.4527 - mean_squared_error: 246.4527
Epoch 40/50
2

2/2 [==============================] - 0s 5ms/step - loss: 1228.4269 - mean_squared_error: 1228.4269
Epoch 27/50
2/2 [==============================] - 0s 3ms/step - loss: 1199.9491 - mean_squared_error: 1199.9491
Epoch 28/50
2/2 [==============================] - 0s 4ms/step - loss: 1187.5784 - mean_squared_error: 1187.5785
Epoch 29/50
2/2 [==============================] - 0s 4ms/step - loss: 1163.4341 - mean_squared_error: 1163.4341
Epoch 30/50
2/2 [==============================] - 0s 4ms/step - loss: 1159.6615 - mean_squared_error: 1159.6615
Epoch 31/50
2/2 [==============================] - 0s 5ms/step - loss: 1157.1682 - mean_squared_error: 1157.1682
Epoch 32/50
2/2 [==============================] - 0s 4ms/step - loss: 1150.9044 - mean_squared_error: 1150.9044
Epoch 33/50
2/2 [==============================] - 0s 3ms/step - loss: 1148.2017 - mean_squared_error: 1148.2018
Epoch 34/50
2/2 [==============================] - 0s 2ms/step - loss: 1145.4375 - mean_squared_error: 1145.

Epoch 23/50
2/2 [==============================] - 0s 4ms/step - loss: 452.9038 - mean_squared_error: 452.9038
Epoch 24/50
2/2 [==============================] - 0s 2ms/step - loss: 452.3562 - mean_squared_error: 452.3562
Epoch 25/50
2/2 [==============================] - 0s 4ms/step - loss: 452.2931 - mean_squared_error: 452.2931
Epoch 26/50
2/2 [==============================] - 0s 2ms/step - loss: 453.0274 - mean_squared_error: 453.0274
Epoch 27/50
2/2 [==============================] - 0s 3ms/step - loss: 451.4973 - mean_squared_error: 451.4973
Epoch 28/50
2/2 [==============================] - 0s 3ms/step - loss: 451.2152 - mean_squared_error: 451.2152
Epoch 29/50
2/2 [==============================] - 0s 4ms/step - loss: 451.3944 - mean_squared_error: 451.3945
Epoch 30/50
2/2 [==============================] - 0s 3ms/step - loss: 450.5013 - mean_squared_error: 450.5013
Epoch 31/50
2/2 [==============================] - 0s 4ms/step - loss: 450.2226 - mean_squared_error: 450.2226
E

# Prediction with aggregated seasonality

A common approach in retail is to consider different coefficients for item products. In this section, all the features, except for the seasonality, are cosidered at product item level inside the dataset. This method is called feature-fixed effect. Note that this method may result in a better prediction for the centralized method, but the decentraized method already considers the dataset at the product item levels. As a result, the prediction accuracy of the decentralized method does not differ from the subsection 2.3.

## Structuring the dataset

In [6]:
sales_fe_sku = sales.copy()
sales_fe_sku = pd.get_dummies(data=sales_fe_sku, columns=['sku'])
sales_fe_sku["sku"] = sales["sku"] 


colnames_to_fix = [i for i in sales.columns if i not in ["week","weekly_sales","sku",
                                                         'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
                                                         'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 
                                                         'month_12']]

sales_seasonality = sales_fe_sku.copy()

for feature in colnames_to_fix:
  for i in range(1,45):
    sales_seasonality[str(feature)+"_fixed_effect_"+str(i)] = sales_seasonality[feature]*sales_seasonality["sku_"+str(i)]

skuSet = list(sales.sku.unique()) #the SKU numbers do not change
skuData = {}
colnames = [i for i in sales_seasonality.columns if i not in ["week","weekly_sales","sku"] and i not in colnames_to_fix]
for i in skuSet:
  df_i = sales_seasonality[sales_seasonality.sku == i]
  skuData[i] = {'X': df_i[:NW][colnames].values,
                'y': df_i[:NW]['weekly_sales'].values}
    


X_dict = {}
y_dict = {}

y_test = []
y_train = []

for i in skuSet:
  
  X_train_i,X_test_i = train_test_split(skuData[i]["X"], shuffle=False, train_size=0.7) #split for X
  y_train_i,y_test_i = train_test_split(skuData[i]["y"], shuffle=False, train_size=0.7) #split for y 

  X_dict[i] = {'train': X_train_i, 'test': X_test_i} #filling dictionary
  y_dict[i] = {'train': y_train_i, 'test': y_test_i}

  y_test += list(y_test_i) 
  y_train += list(y_train_i) 



C:\Users\NOOR~1\AppData\Local\Temp/ipykernel_19340/1985272825.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_seasonality[str(feature)+"_fixed_effect_"+str(i)] = sales_seasonality[feature]*sales_seasonality["sku_"+str(i)]


## Centralized method

In [7]:
X_cen_train = X_dict[skuSet[0]]['train'] #initialization with item 0
X_cen_test = X_dict[skuSet[0]]['test']

for i in skuSet[1:]: #Iteration over items
    X_cen_train = np.concatenate((X_cen_train, X_dict[i]['train']), axis = 0) #Bringing together the training set
    X_cen_test = np.concatenate((X_cen_test, X_dict[i]['test']), axis = 0)

X_cen_train = np.asarray(X_cen_train)
X_cen_test = np.asarray(X_cen_test)

y_test = np.asarray(y_test)
y_train = np.asarray(y_train)

ANN = keras.models.Sequential([
    keras.layers.Dense(5, activation='relu'),
    keras.layers.Dense(5, activation='relu'),
    keras.layers.Dense(1, activation='LeakyReLU')
])

ANN.compile(optimizer=tf.compat.v1.train.AdamOptimizer(), 
              loss='mean_squared_error',
              metrics=['mean_squared_error'])


model_cen = ANN
model_cen.fit(X_cen_train,y_train,batch_size=20,epochs=500,validation_split=0)

y_pred = model_cen.predict(X_cen_test)

print('Centralized method with linear regression R2:',
      round(r2_score(y_test, np.array(y_pred)),3))  
print('Centralized method with linear regression MSE:',
      round(mean_squared_error(y_test, np.array(y_pred)),3))

res['Centralized(SA-DL)']=[r2_score(y_test, model_cen.predict(X_cen_test))]

Epoch 1/500
80/80 [==============================] - 0s 2ms/step - loss: 82161.6797 - mean_squared_error: 82161.6797
Epoch 2/500
80/80 [==============================] - 0s 2ms/step - loss: 81143.7344 - mean_squared_error: 81143.7344
Epoch 3/500
80/80 [==============================] - 0s 2ms/step - loss: 79474.9219 - mean_squared_error: 79474.9219
Epoch 4/500
80/80 [==============================] - 0s 2ms/step - loss: 77107.7422 - mean_squared_error: 77107.7500
Epoch 5/500
80/80 [==============================] - 0s 2ms/step - loss: 74003.6484 - mean_squared_error: 74003.6484
Epoch 6/500
80/80 [==============================] - 0s 2ms/step - loss: 70760.7656 - mean_squared_error: 70760.7656
Epoch 7/500
80/80 [==============================] - 0s 2ms/step - loss: 67850.3906 - mean_squared_error: 67850.3906
Epoch 8/500
80/80 [==============================] - 0s 2ms/step - loss: 65050.7969 - mean_squared_error: 65050.7969
Epoch 9/500
80/80 [==============================] - 0s 2ms/step

80/80 [==============================] - 0s 2ms/step - loss: 28871.2734 - mean_squared_error: 28871.2734
Epoch 140/500
80/80 [==============================] - 0s 1ms/step - loss: 28646.6348 - mean_squared_error: 28646.6348
Epoch 141/500
80/80 [==============================] - 0s 1ms/step - loss: 28440.6660 - mean_squared_error: 28440.6660
Epoch 142/500
80/80 [==============================] - 0s 2ms/step - loss: 28253.3418 - mean_squared_error: 28253.3379
Epoch 143/500
80/80 [==============================] - 0s 1ms/step - loss: 28111.6113 - mean_squared_error: 28111.6113
Epoch 144/500
80/80 [==============================] - 0s 1ms/step - loss: 27924.1484 - mean_squared_error: 27924.1484
Epoch 145/500
80/80 [==============================] - 0s 1ms/step - loss: 27823.0957 - mean_squared_error: 27823.0957
Epoch 146/500
80/80 [==============================] - 0s 1ms/step - loss: 27600.3262 - mean_squared_error: 27600.3262
Epoch 147/500
80/80 [==============================] - 0s 1ms/

Epoch 208/500
80/80 [==============================] - 0s 1ms/step - loss: 17526.3379 - mean_squared_error: 17526.3379
Epoch 209/500
80/80 [==============================] - 0s 2ms/step - loss: 17420.4688 - mean_squared_error: 17420.4688
Epoch 210/500
80/80 [==============================] - 0s 2ms/step - loss: 17318.7344 - mean_squared_error: 17318.7344
Epoch 211/500
80/80 [==============================] - 0s 2ms/step - loss: 17206.1016 - mean_squared_error: 17206.1016
Epoch 212/500
80/80 [==============================] - 0s 2ms/step - loss: 17003.2031 - mean_squared_error: 17003.2031
Epoch 213/500
80/80 [==============================] - 0s 2ms/step - loss: 16877.7695 - mean_squared_error: 16877.7695
Epoch 214/500
80/80 [==============================] - 0s 2ms/step - loss: 16656.7773 - mean_squared_error: 16656.7773
Epoch 215/500
80/80 [==============================] - 0s 2ms/step - loss: 16614.2617 - mean_squared_error: 16614.2617
Epoch 216/500
80/80 [===========================

80/80 [==============================] - 0s 1ms/step - loss: 9992.2549 - mean_squared_error: 9992.2549
Epoch 346/500
80/80 [==============================] - 0s 2ms/step - loss: 9963.9590 - mean_squared_error: 9963.9590
Epoch 347/500
80/80 [==============================] - 0s 1ms/step - loss: 9982.3799 - mean_squared_error: 9982.3799
Epoch 348/500
80/80 [==============================] - 0s 1ms/step - loss: 9942.7080 - mean_squared_error: 9942.7080
Epoch 349/500
80/80 [==============================] - 0s 1ms/step - loss: 9932.0039 - mean_squared_error: 9932.0039
Epoch 350/500
80/80 [==============================] - 0s 1ms/step - loss: 10022.6553 - mean_squared_error: 10022.6553
Epoch 351/500
80/80 [==============================] - 0s 2ms/step - loss: 9954.6328 - mean_squared_error: 9954.6328
Epoch 352/500
80/80 [==============================] - 0s 1ms/step - loss: 9842.6162 - mean_squared_error: 9842.6162
Epoch 353/500
80/80 [==============================] - 0s 1ms/step - loss: 9

80/80 [==============================] - 0s 2ms/step - loss: 7190.1201 - mean_squared_error: 7190.1201
Epoch 486/500
80/80 [==============================] - 0s 2ms/step - loss: 7173.4229 - mean_squared_error: 7173.4229
Epoch 487/500
80/80 [==============================] - 0s 2ms/step - loss: 7099.1118 - mean_squared_error: 7099.1118
Epoch 488/500
80/80 [==============================] - 0s 2ms/step - loss: 7121.5903 - mean_squared_error: 7121.5903
Epoch 489/500
80/80 [==============================] - 0s 2ms/step - loss: 7143.6069 - mean_squared_error: 7143.6069
Epoch 490/500
80/80 [==============================] - 0s 2ms/step - loss: 7150.9907 - mean_squared_error: 7150.9893
Epoch 491/500
80/80 [==============================] - 0s 2ms/step - loss: 7157.3081 - mean_squared_error: 7157.3081
Epoch 492/500
80/80 [==============================] - 0s 2ms/step - loss: 7084.8486 - mean_squared_error: 7084.8486
Epoch 493/500
80/80 [==============================] - 0s 2ms/step - loss: 707

## Decentralized method

In [8]:
y_pred = []
skuModels = {}

for i in skuSet:
 #one model for each item, fitted on training set
 model_i = ANN
 skuModels[i] = model_i.fit(X_dict[i]['train'],y_dict[i]['train'],batch_size=20,epochs=50,validation_split=0)

 #compute and concatenate prediction of the model i on item i
 y_pred += list(model_i.predict(X_dict[i]['test']))


#computing overall performance metrics on y_pred and y_test:
print('Decentralized method with linear regression R2:',round(r2_score(y_test, np.array(y_pred)),3))
print('Decentralized method with linear regression MSE:', round(mean_squared_error(y_test, np.array(y_pred)),3))

res['Decentralized(SA-DL)']=[r2_score(y_test, np.array(y_pred))]

Epoch 1/50
2/2 [==============================] - 0s 4ms/step - loss: 154.6731 - mean_squared_error: 154.6731
Epoch 2/50
2/2 [==============================] - 0s 4ms/step - loss: 141.7352 - mean_squared_error: 141.7352
Epoch 3/50
2/2 [==============================] - 0s 4ms/step - loss: 134.9661 - mean_squared_error: 134.9661
Epoch 4/50
2/2 [==============================] - 0s 8ms/step - loss: 120.2029 - mean_squared_error: 120.2029
Epoch 5/50
2/2 [==============================] - 0s 5ms/step - loss: 126.3322 - mean_squared_error: 126.3322
Epoch 6/50
2/2 [==============================] - 0s 6ms/step - loss: 133.5020 - mean_squared_error: 133.5020
Epoch 7/50
2/2 [==============================] - 0s 3ms/step - loss: 136.1602 - mean_squared_error: 136.1602
Epoch 8/50
2/2 [==============================] - 0s 8ms/step - loss: 118.8169 - mean_squared_error: 118.8169
Epoch 9/50
2/2 [==============================] - 0s 4ms/step - loss: 126.2418 - mean_squared_error: 126.2418
Epoch 10/5

2/2 [==============================] - 0s 3ms/step - loss: 27.8549 - mean_squared_error: 27.8549
Epoch 1/50
2/2 [==============================] - 0s 4ms/step - loss: 44.5952 - mean_squared_error: 44.5952
Epoch 2/50
2/2 [==============================] - 0s 5ms/step - loss: 44.5492 - mean_squared_error: 44.5492
Epoch 3/50
2/2 [==============================] - 0s 3ms/step - loss: 44.5156 - mean_squared_error: 44.5156
Epoch 4/50
2/2 [==============================] - 0s 4ms/step - loss: 44.5060 - mean_squared_error: 44.5060
Epoch 5/50
2/2 [==============================] - 0s 4ms/step - loss: 44.4945 - mean_squared_error: 44.4945
Epoch 6/50
2/2 [==============================] - 0s 4ms/step - loss: 44.4808 - mean_squared_error: 44.4808
Epoch 7/50
2/2 [==============================] - 0s 4ms/step - loss: 44.4626 - mean_squared_error: 44.4626
Epoch 8/50
2/2 [==============================] - 0s 4ms/step - loss: 44.4451 - mean_squared_error: 44.4451
Epoch 9/50
2/2 [=======================

2/2 [==============================] - 0s 4ms/step - loss: 108.5455 - mean_squared_error: 108.5455
Epoch 50/50
2/2 [==============================] - 0s 3ms/step - loss: 108.4410 - mean_squared_error: 108.4410
Epoch 1/50
2/2 [==============================] - 0s 4ms/step - loss: 1236.5568 - mean_squared_error: 1236.5566
Epoch 2/50
2/2 [==============================] - 0s 4ms/step - loss: 1229.2009 - mean_squared_error: 1229.2009
Epoch 3/50
2/2 [==============================] - 0s 5ms/step - loss: 1233.4030 - mean_squared_error: 1233.4030
Epoch 4/50
2/2 [==============================] - 0s 4ms/step - loss: 1229.8568 - mean_squared_error: 1229.8568
Epoch 5/50
2/2 [==============================] - 0s 4ms/step - loss: 1228.3390 - mean_squared_error: 1228.3390
Epoch 6/50
2/2 [==============================] - 0s 4ms/step - loss: 1229.2094 - mean_squared_error: 1229.2094
Epoch 7/50
2/2 [==============================] - 0s 4ms/step - loss: 1225.9111 - mean_squared_error: 1225.9110
Epoch 

2/2 [==============================] - 0s 4ms/step - loss: 1082.6138 - mean_squared_error: 1082.6138
Epoch 45/50
2/2 [==============================] - 0s 4ms/step - loss: 1082.3319 - mean_squared_error: 1082.3319
Epoch 46/50
2/2 [==============================] - 0s 4ms/step - loss: 1082.1053 - mean_squared_error: 1082.1053
Epoch 47/50
2/2 [==============================] - 0s 4ms/step - loss: 1081.7534 - mean_squared_error: 1081.7534
Epoch 48/50
2/2 [==============================] - 0s 4ms/step - loss: 1081.5938 - mean_squared_error: 1081.5938
Epoch 49/50
2/2 [==============================] - 0s 4ms/step - loss: 1081.3392 - mean_squared_error: 1081.3392
Epoch 50/50
2/2 [==============================] - 0s 4ms/step - loss: 1081.1250 - mean_squared_error: 1081.1250
Epoch 1/50
2/2 [==============================] - 0s 4ms/step - loss: 23.7609 - mean_squared_error: 23.7609
Epoch 2/50
2/2 [==============================] - 0s 3ms/step - loss: 23.7688 - mean_squared_error: 23.7688
Epoch

Epoch 42/50
2/2 [==============================] - 0s 4ms/step - loss: 54.9769 - mean_squared_error: 54.9769
Epoch 43/50
2/2 [==============================] - 0s 4ms/step - loss: 54.9445 - mean_squared_error: 54.9445
Epoch 44/50
2/2 [==============================] - 0s 4ms/step - loss: 54.8011 - mean_squared_error: 54.8011
Epoch 45/50
2/2 [==============================] - 0s 3ms/step - loss: 54.7833 - mean_squared_error: 54.7833
Epoch 46/50
2/2 [==============================] - 0s 3ms/step - loss: 54.6521 - mean_squared_error: 54.6521
Epoch 47/50
2/2 [==============================] - 0s 3ms/step - loss: 54.6112 - mean_squared_error: 54.6112
Epoch 48/50
2/2 [==============================] - 0s 11ms/step - loss: 54.5831 - mean_squared_error: 54.5831
Epoch 49/50
2/2 [==============================] - 0s 4ms/step - loss: 54.4114 - mean_squared_error: 54.4114
Epoch 50/50
2/2 [==============================] - 0s 3ms/step - loss: 54.3122 - mean_squared_error: 54.3122
Epoch 1/50
2/2 [==

...

2/2 [==============================] - 0s 4ms/step - loss: 40962.7773 - mean_squared_error: 40962.7773
Epoch 16/50
2/2 [==============================] - 0s 4ms/step - loss: 40549.0625 - mean_squared_error: 40549.0625
Epoch 17/50
2/2 [==============================] - 0s 4ms/step - loss: 40250.9219 - mean_squared_error: 40250.9219
Epoch 18/50
2/2 [==============================] - 0s 4ms/step - loss: 40156.1602 - mean_squared_error: 40156.1602
Epoch 19/50
2/2 [==============================] - 0s 3ms/step - loss: 40024.0430 - mean_squared_error: 40024.0430
Epoch 20/50
2/2 [==============================] - 0s 4ms/step - loss: 39927.9180 - mean_squared_error: 39927.9180
Epoch 21/50
2/2 [==============================] - 0s 2ms/step - loss: 39668.0781 - mean_squared_error: 39668.0781
Epoch 22/50
2/2 [==============================] - 0s 4ms/step - loss: 40169.3750 - mean_squared_error: 40169.3750
Epoch 23/50
2/2 [==============================] - 0s 3ms/step - loss: 39259.2656 - mean_squ

Epoch 12/50
2/2 [==============================] - 0s 4ms/step - loss: 112.7103 - mean_squared_error: 112.7103
Epoch 13/50
2/2 [==============================] - 0s 4ms/step - loss: 107.3445 - mean_squared_error: 107.3445
Epoch 14/50
2/2 [==============================] - 0s 4ms/step - loss: 105.0991 - mean_squared_error: 105.0991
Epoch 15/50
2/2 [==============================] - 0s 3ms/step - loss: 105.4979 - mean_squared_error: 105.4979
Epoch 16/50
2/2 [==============================] - 0s 4ms/step - loss: 107.6091 - mean_squared_error: 107.6091
Epoch 17/50
2/2 [==============================] - 0s 4ms/step - loss: 106.5797 - mean_squared_error: 106.5797
Epoch 18/50
2/2 [==============================] - 0s 4ms/step - loss: 103.7273 - mean_squared_error: 103.7273
Epoch 19/50
2/2 [==============================] - 0s 4ms/step - loss: 103.1069 - mean_squared_error: 103.1069
Epoch 20/50
2/2 [==============================] - 0s 4ms/step - loss: 103.6479 - mean_squared_error: 103.6479
E

2/2 [==============================] - 0s 3ms/step - loss: 202.7146 - mean_squared_error: 202.7146
Epoch 8/50
2/2 [==============================] - 0s 3ms/step - loss: 201.6567 - mean_squared_error: 201.6567
Epoch 9/50
2/2 [==============================] - 0s 3ms/step - loss: 200.9343 - mean_squared_error: 200.9343
Epoch 10/50
2/2 [==============================] - 0s 2ms/step - loss: 203.8869 - mean_squared_error: 203.8869
Epoch 11/50
2/2 [==============================] - 0s 4ms/step - loss: 204.5202 - mean_squared_error: 204.5202
Epoch 12/50
2/2 [==============================] - 0s 3ms/step - loss: 205.9103 - mean_squared_error: 205.9103
Epoch 13/50
2/2 [==============================] - 0s 3ms/step - loss: 206.0522 - mean_squared_error: 206.0522
Epoch 14/50
2/2 [==============================] - 0s 4ms/step - loss: 204.9577 - mean_squared_error: 204.9577
Epoch 15/50
2/2 [==============================] - 0s 4ms/step - loss: 203.5782 - mean_squared_error: 203.5782
Epoch 16/50
2/2

2/2 [==============================] - 0s 3ms/step - loss: 20.9063 - mean_squared_error: 20.9063
Epoch 7/50
2/2 [==============================] - 0s 4ms/step - loss: 20.8779 - mean_squared_error: 20.8779
Epoch 8/50
2/2 [==============================] - 0s 3ms/step - loss: 20.8531 - mean_squared_error: 20.8531
Epoch 9/50
2/2 [==============================] - 0s 3ms/step - loss: 20.8747 - mean_squared_error: 20.8747
Epoch 10/50
2/2 [==============================] - 0s 3ms/step - loss: 20.8322 - mean_squared_error: 20.8322
Epoch 11/50
2/2 [==============================] - 0s 4ms/step - loss: 20.8551 - mean_squared_error: 20.8551
Epoch 12/50
2/2 [==============================] - 0s 4ms/step - loss: 20.8375 - mean_squared_error: 20.8375
Epoch 13/50
2/2 [==============================] - 0s 4ms/step - loss: 20.8115 - mean_squared_error: 20.8115
Epoch 14/50
2/2 [==============================] - 0s 4ms/step - loss: 20.8092 - mean_squared_error: 20.8092
Epoch 15/50
2/2 [=================

2/2 [==============================] - 0s 4ms/step - loss: 12.1424 - mean_squared_error: 12.1424
Epoch 4/50
2/2 [==============================] - 0s 3ms/step - loss: 11.3357 - mean_squared_error: 11.3357
Epoch 5/50
2/2 [==============================] - 0s 4ms/step - loss: 10.7759 - mean_squared_error: 10.7759
Epoch 6/50
2/2 [==============================] - 0s 4ms/step - loss: 10.3416 - mean_squared_error: 10.3416
Epoch 7/50
2/2 [==============================] - 0s 3ms/step - loss: 10.1910 - mean_squared_error: 10.1910
Epoch 8/50
2/2 [==============================] - 0s 3ms/step - loss: 10.2954 - mean_squared_error: 10.2954
Epoch 9/50
2/2 [==============================] - 0s 4ms/step - loss: 10.3854 - mean_squared_error: 10.3854
Epoch 10/50
2/2 [==============================] - 0s 3ms/step - loss: 10.5019 - mean_squared_error: 10.5019
Epoch 11/50
2/2 [==============================] - 0s 4ms/step - loss: 10.5440 - mean_squared_error: 10.5440
Epoch 12/50
2/2 [====================

# Results

In [9]:
res

,Centralized(DL),Decentralized(DL),Centralized(SA-DL),Decentralized(SA-DL)
R2,0.354207,0.401126,0.348458,0.353766


When we have the data for only a single year, the performance of the deep learning reduces significantly. The best approach here is to use the decentralized approach without seasonality aggregation.